#### City Bike Analytics Workbook

In [1]:
import pandas as pd

In [2]:
# specify data path
jan = "../Tableau-CitiBike-Visualization/Data/JC-202101-citibike-tripdata.csv"
feb = "../Tableau-CitiBike-Visualization/Data/JC-202102-citibike-tripdata.csv"
mar = "../Tableau-CitiBike-Visualization/Data/JC-202103-citibike-tripdata.csv"

# read csv file into jupyter notebook
# Examine column count and compare data headers with other files for consistency
jan_data = pd.read_csv(jan)
feb_data = pd.read_csv(feb)
mar_data = pd.read_csv(mar)
jan_data.head(1)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,266,2021-01-01 00:03:35.5100,2021-01-01 00:08:01.7770,3273,Manila & 1st,40.721651,-74.042884,3209,Brunswick St,40.724176,-74.050656,42494,Subscriber,1988,1


In [3]:
# Examine df head
feb_data = pd.read_csv(feb)
feb_data.head(1)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,121DD7DD23CB1335,docked_bike,2021-02-03 23:11:28,2021-02-03 23:18:28,Hoboken Ave at Monmouth St,JC105,Christ Hospital,JC034,40.735208,-74.046964,40.734786,-74.050444,member


In [4]:
# Examine df head
mar_data = pd.read_csv(mar)
mar_data.head(1)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,ABBBB4D89C620898,docked_bike,2021-03-03 10:32:07,2021-03-03 10:32:40,Jackson Square,JC063,Jackson Square,JC063,40.71113,-74.0789,40.71113,-74.0789,member


In [5]:
# Check number of columns
print(len(jan_data.columns))
print(len(feb_data.columns))
print(len(mar_data.columns))

15
13
13


In [6]:
# combine jan & feb df into one df since the df heads are the same
combine_jan_feb_df = pd.concat([jan_data, feb_data])

# combine_jan_feb_df.head(1)  #used to check df, commented out

# check number of rows in index after concat
print(len(combine_jan_feb_df.index))

16505


C:\Users\jchan\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [7]:
# rename column headers
rename_janfeb_df = combine_jan_feb_df
rename_janfeb_df.rename(columns={"starttime":"started_at",
                        "stoptime":"ended_at",
                         "start station id":"start_station_id",
                         "end station id":"end_station_id",
                         "start station name":"start_station_name",
                         "end station name":"end_station_name",
                         "start station latitude":"start_lat",
                         "end station latitude":"end_lat",
                         "start station longitude":"start_lng",
                         "end station longitude":"end_lng",
                        "start station name":"start_station_name",
                        }, inplace=True)
rename_janfeb_df.head()

,bikeid,birth year,end_station_id,end_lat,end_lng,end_station_name,end_lat,end_lng,end_station_id,end_station_name,...,start_station_name,start_lat,start_lng,start_station_id,start_station_name,started_at,started_at,ended_at,tripduration,usertype
0,42494.0,1988.0,3209.0,40.724176,-74.050656,Brunswick St,NaN,NaN,NaN,NaN,...,Manila & 1st,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:03:35.5100,2021-01-01 00:08:01.7770,266.0,Subscriber
1,45343.0,1996.0,3213.0,40.718489,-74.047727,Van Vorst Park,NaN,NaN,NaN,NaN,...,Grand St,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:23:32.9250,2021-01-01 00:49:16.0830,1543.0,Customer
2,31794.0,1995.0,3213.0,40.718489,-74.047727,Van Vorst Park,NaN,NaN,NaN,NaN,...,Grand St,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:23:50.7940,2021-01-01 00:48:12.5660,1461.0,Customer
3,42316.0,1969.0,3199.0,40.728745,-74.032108,Newport Pkwy,NaN,NaN,NaN,NaN,...,City Hall,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:31:09.0770,2021-01-01 00:44:22.9430,793.0,Customer
4,32575.0,1969.0,3209.0,40.724176,-74.050656,Brunswick St,NaN,NaN,NaN,NaN,...,Harborside,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:35:52.1900,2021-01-01 00:45:48.7740,596.0,Customer


In [8]:
rename_janfeb_df.drop(["usertype", "bikeid", "birth year", "gender"], axis=1, inplace=True)
rename_janfeb_df.head()

,end_station_id,end_lat,end_lng,end_station_name,end_lat,end_lng,end_station_id,end_station_name,ended_at,member_casual,...,start_lng,start_station_name,start_lat,start_lng,start_station_id,start_station_name,started_at,started_at,ended_at,tripduration
0,3209.0,40.724176,-74.050656,Brunswick St,NaN,NaN,NaN,NaN,NaN,NaN,...,-74.042884,Manila & 1st,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:03:35.5100,2021-01-01 00:08:01.7770,266.0
1,3213.0,40.718489,-74.047727,Van Vorst Park,NaN,NaN,NaN,NaN,NaN,NaN,...,-74.037683,Grand St,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:23:32.9250,2021-01-01 00:49:16.0830,1543.0
2,3213.0,40.718489,-74.047727,Van Vorst Park,NaN,NaN,NaN,NaN,NaN,NaN,...,-74.037683,Grand St,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:23:50.7940,2021-01-01 00:48:12.5660,1461.0
3,3199.0,40.728745,-74.032108,Newport Pkwy,NaN,NaN,NaN,NaN,NaN,NaN,...,-74.043845,City Hall,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:31:09.0770,2021-01-01 00:44:22.9430,793.0
4,3209.0,40.724176,-74.050656,Brunswick St,NaN,NaN,NaN,NaN,NaN,NaN,...,-74.034234,Harborside,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:35:52.1900,2021-01-01 00:45:48.7740,596.0


In [9]:
df = rename_janfeb_df

# Convert trip duration column from seconds to minutes
df["tripduration"] = [trip/60 for trip in df["tripduration"]]

# Filter out trips over 24 hours long
df = df.loc[df["tripduration"] < 1440, :]

# Rename trip duration column
df.columns = df.columns.str.replace("tripduration", "tripduration (min)")

df.head()

,end_station_id,end_lat,end_lng,end_station_name,end_lat,end_lng,end_station_id,end_station_name,ended_at,member_casual,...,start_lng,start_station_name,start_lat,start_lng,start_station_id,start_station_name,started_at,started_at,ended_at,tripduration (min)
0,3209.0,40.724176,-74.050656,Brunswick St,NaN,NaN,NaN,NaN,NaN,NaN,...,-74.042884,Manila & 1st,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:03:35.5100,2021-01-01 00:08:01.7770,4.433333
1,3213.0,40.718489,-74.047727,Van Vorst Park,NaN,NaN,NaN,NaN,NaN,NaN,...,-74.037683,Grand St,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:23:32.9250,2021-01-01 00:49:16.0830,25.716667
2,3213.0,40.718489,-74.047727,Van Vorst Park,NaN,NaN,NaN,NaN,NaN,NaN,...,-74.037683,Grand St,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:23:50.7940,2021-01-01 00:48:12.5660,24.350000
3,3199.0,40.728745,-74.032108,Newport Pkwy,NaN,NaN,NaN,NaN,NaN,NaN,...,-74.043845,City Hall,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:31:09.0770,2021-01-01 00:44:22.9430,13.216667
4,3209.0,40.724176,-74.050656,Brunswick St,NaN,NaN,NaN,NaN,NaN,NaN,...,-74.034234,Harborside,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:35:52.1900,2021-01-01 00:45:48.7740,9.933333


In [10]:
# Add ride id column to uniquely identify each ride
df["rideid"] = df["start_station_id"].map(str) + "_" + df["end_station_id"].map(str)
df.head()

AttributeError: 'DataFrame' object has no attribute 'map'

In [ ]:
# break out year
df['trip_Year'] = df.started_at.str.split("-", expand=True)[0]

# break out month
df['trip_month'] = df.started_at.str.split("-", expand=True)[1]
df.head(25)

In [ ]:
df["trip_month"].replace({"01": "January", "02": "February", "03": "March"}, inplace=True)
df.head()

###### Calculate distance of each ride point a to b.

In [ ]:


from math import sin, cos, sqrt, atan2, radians

# For lat and lon, convert degrees to radians
lat1 = [radians(lat) for lat in df["start_lat"]]
lon1 = [radians(lon) for lon in df["start_lng"]]
lat2 = [radians(lat) for lat in df["end_lat"]]
lon2 = [radians(lon) for lon in df["end_lng"]]

# Convert lists into series
lat1 = pd.Series(lat1)
lon1 = pd.Series(lon1)
lat2 = pd.Series(lat2)
lon2 = pd.Series(lon2)

# Calculate difference between each set of latitude and longitude
dlat = lat2 - lat1
dlon = lon2 - lon1

In [ ]:
# Approximate radius of Earth in km
R = 6373.0

# Empty list to store trip distances
distance = []

for i in range(0, len(lat1)):
    
    a = sin(dlat[i] / 2)**2 + cos(lat1[i]) * cos(lat2[i]) * sin(dlon[i] / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Get distance and convert km to miles
    miles = (R * c) * .6214
    
    # Append miles travel to 'distance' list
    distance.append(miles)

In [ ]:
# Add trip distance as new column to data frame
df.loc[:, "tripdistance (mi)"] = distance

# df["tripdistance (mi)"] = distance
df.head()

In [ ]:
df.to_csv("../Tableau-CitiBike-Visualization/Output/upload_to_tableau.csv")

In [ ]:
# https://hackersandslackers.com/splitting-columns-with-pandas/